This notebook is dedicated to the First Part of the First Project within IBM's Applied Data Science Capstone Course. In this notebook, now that I have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. 
Notebook Creator: Arnav Mangal, Date: 1/24/2021

In [1]:
#import urllib for opening URL
import urllib.request

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

#page variable will have HTML of wiki page
page = urllib.request.urlopen(url)

In [2]:
#import BeautifulSoup Library to parse HTML and XML docs
from bs4 import BeautifulSoup

soup = BeautifulSoup(page, "lxml")

In [3]:
#print(soup.prettify())

In [4]:
#soup.title.string  >> Displays title of Wiki Page
all_tables=soup.find_all("table")
#all_tables

In [5]:
right_table=soup.find('table', class_='wikitable sortable')
#right_table


In [6]:
A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))


In [7]:
import pandas as pd
df4 = pd.DataFrame(A, columns = ['PostalCode'])
df4['Borough']=B
df4['Neighbourhood']=C
df4

,PostalCode,Borough,Neighbourhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
...,...,...,...
175,M5Z\n,Not assigned\n,Not assigned\n
176,M6Z\n,Not assigned\n,Not assigned\n
177,M7Z\n,Not assigned\n,Not assigned\n
178,M8Z\n,Etobicoke\n,"Mimico NW, The Queensway West, South of Bloor,..."


In [8]:
df4['Borough'] =  df4['Borough'].str.rstrip('\n')
df4['PostalCode'] =  df4['PostalCode'].str.rstrip('\n')
df4['Neighbourhood'] =  df4['Neighbourhood'].str.rstrip('\n')
df4

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [9]:
#Now, Drop any Boroughs with "not assigned"
# Get indexes where Borough column has value "Not assigned"


import numpy as np
df4.replace("Not assigned", np.nan, inplace = True)

df4

,PostalCode,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,NaN,NaN
176,M6Z,NaN,NaN
177,M7Z,NaN,NaN
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [10]:
# simply drop whole row with NaN in "price" column
df4.dropna(subset=["Borough"], axis=0, inplace=True)

# reset index, because we droped two rows
df4.reset_index(drop=True, inplace=True)
df4.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [11]:
df4.shape

(103, 3)

Part 2: Finding Latitude and Longitude

In [18]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.columns = ['PostalCode', 'Latitude', 'Longitude']

df_pos = pd.merge(df4, df_geo, on=['PostalCode'], how='inner')
df_pos

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [20]:
df_pos.shape

(103, 5)